In [1]:
import nbformat as nbf
from pathlib import Path

nb = nbf.v4.new_notebook()
nb["metadata"]["colab"] = {"provenance": []}
cells = []

# Title
cells.append(nbf.v4.new_markdown_cell(
"# 02 — Portfolio: Rolling Rebalance (Max-Sharpe, Monthly) + Metrics\n"
"ขั้นตอน: โหลดราคา → คำนวณผลตอบแทนรายวัน → Rolling window 252 วัน → สร้างน้ำหนัก Max-Sharpe รายเดือน → ผลตอบแทนกลยุทธ์ → Metrics (Sharpe, CAGR, MaxDD, Vol) + เทียบ Equal-Weight"
))

# Setup
cells.append(nbf.v4.new_code_cell("""\
%pip -q install yfinance pandas numpy matplotlib scikit-learn scipy
"""))

# Imports + Params + Download
cells.append(nbf.v4.new_code_cell("""\
import yfinance as yf, pandas as pd, numpy as np, matplotlib.pyplot as plt, os
from sklearn.covariance import LedoitWolf
from scipy.optimize import minimize

# ---------- Parameters ----------
TICKERS = ["PTT.BK","AOT.BK","SCB.BK","CPALL.BK","ADVANC.BK"]  # แก้ได้
START = "2019-01-01"
END   = "2024-12-31"
LOOKBACK_DAYS = 252   # window สำหรับคำนวณ mean/cov
REBAL_FREQ = "M"      # รายเดือน
RF_ANNUAL = 0.0       # risk-free สำหรับ Sharpe (ปรับตามมุมมอง)

# ---------- Load prices ----------
px = yf.download(TICKERS, start=START, end=END, auto_adjust=False)["Adj Close"].ffill().dropna(how="all")
rets = px.pct_change().dropna()
print("Prices:", px.shape, "Returns:", rets.shape)
px.tail(3)
"""))

# Utilities (optimizer + metrics)
cells.append(nbf.v4.new_code_cell("""\
from typing import Dict

def max_sharpe_weights(rw: pd.DataFrame, rf_annual=0.0) -> np.ndarray:
    \"\"\"คำนวณน้ำหนัก Max-Sharpe จากหน้าต่าง rolling (daily returns DataFrame)\"\"\"
    n = rw.shape[1]
    mu_ann = rw.mean().values * 252
    cov_ann = LedoitWolf().fit(rw.values).covariance_ * 252
    bounds = [(0,1)] * n
    cons = [{"type":"eq","fun": lambda w: np.sum(w) - 1}]
    def neg_sharpe(w):
        w = np.asarray(w)
        ret = w @ mu_ann
        vol = np.sqrt(w @ cov_ann @ w) + 1e-12
        return -( (ret - rf_annual) / vol )
    x0 = np.ones(n)/n
    res = minimize(neg_sharpe, x0, method="SLSQP", bounds=bounds, constraints=cons, options={"maxiter":200})
    return res.x

def perf_metrics(returns: pd.Series, periods_per_year=252, rf=0.0) -> Dict[str, float]:
    r = returns.dropna()
    if len(r) == 0:
        return {"CAGR": np.nan, "Vol": np.nan, "Sharpe": np.nan, "MaxDD": np.nan}
    cum = (1 + r).cumprod()
    years = len(r) / periods_per_year
    cagr = float(cum.iloc[-1]**(1/years) - 1) if years > 0 else np.nan
    vol = float(r.std() * np.sqrt(periods_per_year))
    sharpe = float((r.mean()*periods_per_year - rf) / (vol + 1e-12))
    roll_max = cum.cummax()
    dd = (cum/roll_max - 1.0)
    mdd = float(dd.min())
    return {"CAGR": cagr, "Vol": vol, "Sharpe": sharpe, "MaxDD": mdd}
"""))

# Rolling Rebalance Loop
cells.append(nbf.v4.new_code_cell("""\
# ---------- Rebalance dates ----------
rebal_dates = rets.resample(REBAL_FREQ).last().index

weights_records = []
port_rets = pd.Series(dtype=float)  # จะต่อซีรีส์ผลตอบแทนรายวันของพอร์ต

for d in rebal_dates:
    # ให้ d_eff เป็นวันที่เทรดจริงที่ใกล้ที่สุด (ถ้าตรงวันหยุด จะเลื่อนไปวันก่อนหน้า)
    if d in rets.index:
        d_eff = d
    else:
        pos = rets.index.searchsorted(d, side="right") - 1
        if pos <= 0:
            continue
        d_eff = rets.index[pos]

    # เลือกหน้าต่างย้อนหลัง LOOKBACK_DAYS วันก่อน d_eff
    end_loc = rets.index.get_loc(d_eff)
    start_loc = end_loc - LOOKBACK_DAYS
    if start_loc <= 0:
        continue
    rw = rets.iloc[start_loc:end_loc]  # ข้อมูลอดีตเท่านั้น

    # หา weights แบบ Max Sharpe
    w = max_sharpe_weights(rw, rf_annual=RF_ANNUAL)
    weights_records.append(pd.Series(w, index=rets.columns, name=d_eff))

    # ช่วงถือครอง: ตั้งแต่วันถัดจาก d_eff จนถึงวัน rebal ถัดไป (รวมวันสุดท้าย)
    try:
        next_d = rebal_dates[rebal_dates.get_loc(d)+1]
    except Exception:
        next_d = rets.index[-1]
    # แปลง next_d เป็นวันเทรดจริง
    if next_d in rets.index:
        next_eff = next_d
    else:
        pos2 = rets.index.searchsorted(next_d, side="right") - 1
        next_eff = rets.index[max(pos2, end_loc+1)]

    hold = rets.loc[(rets.index > d_eff) & (rets.index <= next_eff)]
    if len(hold) == 0:
        continue
    daily_port = hold @ w  # ผลตอบแทนพอร์ตแบบ dot weight
    port_rets = pd.concat([port_rets, daily_port])

# ---------- Save outputs ----------
weights_df = pd.DataFrame(weights_records)
weights_df.to_csv("rolling_weights_max_sharpe.csv")
port_rets.to_csv("rolling_rebalance_returns.csv", header=["ret"])
print("Saved: rolling_weights_max_sharpe.csv", weights_df.shape)
print("Saved: rolling_rebalance_returns.csv", port_rets.shape)
"""))

# Benchmarks + Metrics + Plot
cells.append(nbf.v4.new_code_cell("""\
# Benchmark: Equal-Weight (ถือเท่ากันทุกวัน)
ew = rets.mean(axis=1)  # proxy agg (เท่ากันทุกตัว, ไม่ rebal รายวันละเอียด)
# คุณสามารถทำ EW รายเดือนแบบคำนวณ weights รายเดือนคล้าย Max-Sharpe ได้เช่นกัน

# Metrics
met_roll = perf_metrics(port_rets, periods_per_year=252, rf=RF_ANNUAL)
met_ew   = perf_metrics(ew.reindex(port_rets.index).dropna(), periods_per_year=252, rf=RF_ANNUAL)

df_mets = pd.DataFrame([
    {"Strategy":"Rolling_MaxSharpe", **met_roll},
    {"Strategy":"EqualWeight_proxy", **met_ew},
])
df_mets.to_csv("strategy_performance_metrics.csv", index=False)
print("Saved: strategy_performance_metrics.csv")
display(df_mets)

# Plot cumulative returns
ax = (port_rets.cumsum()).plot(figsize=(10,5), label="Rolling MaxSharpe")
ew.reindex(port_rets.index).cumsum().plot(ax=ax, label="EW proxy")
plt.title("Rolling Rebalance — Cumulative Sum of Daily Returns")
plt.legend(); plt.show()
"""))

nb["cells"] = cells
Path("02_mpt_rolling_rebalance.ipynb").write_text(nbf.writes(nb), encoding="utf-8")
print("✅ Created 02_mpt_rolling_rebalance.ipynb")


✅ Created 02_mpt_rolling_rebalance.ipynb
